In [16]:
#-*- coding:utf-8 -*-
import folium
import os
import branca
import pandas as pd

lib = pd.read_csv("libGeolocation.csv", engine="python")

x = lib[["도서관명", "주소", "위도", "경도", "전화번호", "홈페이지"]]

xlat = x.loc[0, "위도"]

xlon = x.loc[0, "경도"]

x.shape[0]

x.head()

,도서관명,주소,위도,경도,전화번호,홈페이지
0,강릉교육문화관,강원도 강릉시 노암등길 37-1,37.745942,128.900005,033-640-9911,http://lib.gwe.go.kr/kanglib/
1,명주교육도서관,강원도 강릉시 주문진읍 해안3길 11,37.886773,128.826391,033-660-6901,http://lib.gwe.go.kr/mjlib/
2,고성교육도서관,강원도 고성군 거진읍 거탄진로29번길 7,38.439261,128.450089,033-682-4886,http://lib.gwe.go.kr/gosung/
3,동해교육도서관,강원도 동해시 부곡5길 22,37.538486,129.109787,033-533-4436,http://lib.gwe.go.kr/donglib/
4,삼척교육문화관,강원도 삼척시 진주로 48-41,37.445145,129.165007,033-570-5543,http://lib.gwe.go.kr/samleic/


In [17]:
import numpy as np

lib = pd.read_csv("참여도서관 수.csv", engine = "python")

lib.head()

do_lib = pd.pivot_table(lib, index='도', aggfunc=np.sum)
do_lib['참여도서관 수']

도
Busan                 21
Chungcheongbuk-do     36
Chungcheongnam-do     23
Daegu                 51
Daejeon               25
Gangwon-do            23
Gwangju               10
Gyeonggi-do          231
Gyeongsangbuk-do      50
Gyeongsangnam-do      62
Incheon               14
Jeju-do               15
Jeollabuk-do          22
Jeollanam-do          36
Sejongsi               2
Seoul                198
Ulsan                 26
Name: 참여도서관 수, dtype: int64

In [18]:
import requests
import json

recvd = requests.get("http://ip.jsontest.com/")
my_ip = json.loads(recvd.text)
print(my_ip['ip'])

#출처: https://pythonkim.tistory.com/99 [파이쿵]

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [11]:
import geoip2.database
reader = geoip2.database.Reader('GeoLite2-City.mmdb')
response = reader.city(my_ip['ip'])
response.location.latitude
response.location.longitude

user_lat = response.location.latitude
user_lon = response.location.longitude

NameError: name 'my_ip' is not defined

In [27]:
import warnings
from folium.plugins import MarkerCluster, Search, MousePosition, FloatImage, Draw, MeasureControl
from folium import FeatureGroup, LayerControl, plugins

warnings.simplefilter(action = "ignore", category = FutureWarning)
 
geo_path = 'skorea-provinces-2018-geo.json'
geo_json = json.load(open(geo_path, encoding='utf-8'))

#지도위에 마커 표시하기
def lib_spot():
    #지도 생성
    lib_map = folium.Map(location=[36, 128], zoom_start = 7, control_scale=True)
    
    #마커 클러스터 생성
    marker_cluster = MarkerCluster(name = "도서관 Cluster").add_to(lib_map)
    
    #feature group 생성
    #feature_group = FeatureGroup(name='도서관 Marker')
    
    #참여도서관 수 choropleth로 표현
    lib_map.choropleth(geo_data = geo_json,
                       data = do_lib['참여도서관 수'],
                       columns = [do_lib.index, do_lib['참여도서관 수']],
                       key_on = 'feature.properties.name_eng',
                       fill_color = 'Greens',
                       fill_opacity = 0.7,
                       line_opacity = 0.2,
                       legend_name = "참여도서관 수",
                       highlight = True,
                       name = '단계구분도'
                      )
    
    #지도 위에 도서관 위치 표시
    length = x.shape[0]
    
    for i in range(0, length, 1):
        #마커 표시를 위한 데이터 생성
        lat = x.loc[i, "위도"]
        lon = x.loc[i, "경도"]
        lib_name = str(x.loc[i, "도서관명"])
        lib_add = str(x.loc[i, "주소"])
        lib_url = str("<a href=\"{}\">{}<a>").format(x.loc[i, "홈페이지"], x.loc[i, "홈페이지"])
        lib_call = str(x.loc[i, "전화번호"])
        popup_msg = ("<b>도서관명:<b> {name}<br>""<b>주소:<b> {address}</a> <br>""<b>URL:<b> {url}</a> <br>""<b>전화번호:<b> {call} <br>").format(name = lib_name, address = lib_add, url = lib_url, call = lib_call)
        
        #마커 표시   
        #folium.CircleMarker([lat, lon], radius = 5, popup = popup_msg, name = lib_name, fill = True, fill_color = "blue", color = "blue").add_to(feature_group)
        
        folium.CircleMarker([lat, lon], radius = 5, popup = popup_msg, name = lib_name, fill = True, fill_color = "skyblue", color = "blue", fill_opacity= 0.5, tooltip = x.loc[i, "도서관명"]).add_to(marker_cluster)
    
    #feature group 추가
    #feature_group.add_to(lib_map)
    
    #IP를 통해 이용자가 접속한 위치 표시
    #user_msg = "Your current location."
    #user_location = folium.FeatureGroup(name='당신의 현재 위치(IP 기반)').add_to(lib_map)
    #folium.Marker([user_lat, user_lon], popup = user_msg, icon = folium.Icon(color="red", icon="info-sign")).add_to(user_location)
    
    #미니맵 추가
    minimap = plugins.MiniMap()
    lib_map.add_child(minimap)
    
    #tile 종류 추가
    folium.TileLayer('Mapbox Bright').add_to(lib_map)
    folium.TileLayer('Mapbox Control Room').add_to(lib_map)
    folium.TileLayer('stamenterrain').add_to(lib_map)
    folium.TileLayer('stamentoner').add_to(lib_map)
    folium.TileLayer('stamenwatercolor').add_to(lib_map)
    folium.TileLayer('cartodbpositron').add_to(lib_map)
    folium.TileLayer('cartodbdark_matter').add_to(lib_map)
    
    #구글맵 추가
    folium.raster_layers.TileLayer(
        tiles='http://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr='google',
        name='google maps',
        max_zoom=20,
        subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
        overlay=False,
        control=True,
    ).add_to(lib_map)

    folium.raster_layers.TileLayer(
        tiles='http://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr='google',
        name='google street view',
        max_zoom=20,
        subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
        overlay=False,
        control=True,
    ).add_to(lib_map)
    
    #tile layer 추가
    LayerControl().add_to(lib_map)

    #mouse position 추가
    formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
    MousePosition(position='bottomleft',
                  separator=' | ',
                  empty_string='NaN',
                  prefix = '좌표',
                  lat_formatter=formatter,
                  lng_formatter=formatter).add_to(lib_map)
    
    #full screen 추가
    plugins.Fullscreen(
        position='topleft',
        title='전체화면으로 표시',
        title_cancel='돌아가기',
        force_separate_button=True
    ).add_to(lib_map)
    
    #그림 그리기 추가
    #draw = Draw()
    #draw.add_to(lib_map)
    
    #measure control 추가
    measure = MeasureControl(position = 'topleft')
    measure.add_to(lib_map)
    
    #HTML 파일로 저장
    lib_map.save("lib_map.html")

lib_spot()

In [24]:
testMap = folium.Map(location=[36, 124], zoom_start = 8)
p1 = [x.loc[753, "위도"], x.loc[753, "경도"]]
p2 = [x.loc[754, "위도"], x.loc[754, "경도"]]
p3 = [x.loc[755, "위도"], x.loc[755, "경도"]]
p4 = [x.loc[756, "위도"], x.loc[756, "경도"]]
p5 = [x.loc[757, "위도"], x.loc[757, "경도"]]
p6 = [x.loc[758, "위도"], x.loc[758, "경도"]]
p7 = [x.loc[759, "위도"], x.loc[759, "경도"]]
p8 = [x.loc[760, "위도"], x.loc[760, "경도"]]

pArray = [p1, p2, p3, p4, p5, p6, p7, p8]

for i in range(0, 7, 1):
    #folium.PolyLine([pArray[i], pArray[i]], interactive = True, bubblingMouseEvents = True, color='red', weight = 3).add_to(testMap)
    for j in range(0, 7, 1):
        folium.PolyLine([pArray[i], pArray[j]], lineCap = 'round', lineJoin = 'round', color='red', weight = 2).add_to(testMap)
        #olium.PolyLine([p1, p2, p3, p4, p5, p6, p7, p8], lineCap = 'round', lineJoin = 'round', color='red', weight = 2).add_to(testMap)

#popupLine = folium.ColorLine([pArray[0], pArray[1]], colors='red', weight = 2)

#folium.CircleMarker(pArray[0], fill = True, fill_color = "blue", color = "skyblue", target = popupLine).add_to(testMap)
        
testMap